## 2. 벡터 저장소 (Vector Store)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [ ]:
from langchain_elasticsearch import ElasticsearchStore

vector_poi = "vector_poi_v2"

vector_store = ElasticsearchStore(
    embedding=embeddings_model,
    index_name=vector_poi,
    es_url="http://localhost:9200"
)

In [ ]:
import pandas as pd

fielePath = "./data/poi_data.csv"

def read_excel_as_list(file_path):
    # Excel 파일 읽기
    try:
        # pandas로 Excel 파일 읽기
        df = pd.read_csv(file_path, encoding='utf-8')
        
        # DataFrame을 리스트로 변환
        data_list = df.values.tolist()

        return data_list

    except Exception as e:
        print(f"Error reading the Excel file: {e}")
        return []

data_list = read_excel_as_list(file_path=fielePath)
print(data_list[1])
for i in range(len(data_list[1])):
    print(i , data_list[1][i])
print(len(data_list))

In [ ]:
import math

a    = 6378137.0               
f    = 1 / 298.257222101      
lat0 = 38.0 * math.pi / 180.0  
lon0 = 127.0 * math.pi / 180.0 
k0   = 1.0                     
x0   = 200000.0                
y0   = 500000.0   

def tm_to_wgs84(x, y):

    math.sqrt(2 * f- f * f)
    n = f / (2 - f)
    A = a / (1 + n) * (1 + n*n/4 + n*n*n*n/64)

    x = x - x0
    y = y - y0

    lat = lat0 
    for i in range(5):
        lat = (y / (k0 * A)) + lat0
	
    lon = lon0 + (x / (k0 * A * math.cos(lat)))
    lat_deg = lat * 180.0 / math.pi
    lon_deg = lon * 180.0 / math.pi

    return lon_deg, lat_deg

# embedding field using embedded model 


In [ ]:
# declare variance
es_url = "http://localhost:9200"
es_index_name = "vector_poi_v2"

mapping = {
    "mappings": {
        "properties": {
            "title_vector": {  # title 벡터 필드
                "type": "dense_vector",
                "dims": 1024  # Hugging Face 모델에서 생성된 벡터 차원 수
            },
            "address_vector": {  # address 벡터 필드
                "type": "dense_vector",
                "dims": 1024
            },
            "location": {  # lat, lon을 포함하는 geo_point 필드
                "type": "geo_point"
            },
            "title": {  # title 원본 텍스트
                "type": "text"
            },
            "address": {  # address 원본 텍스트
                "type": "text"
            },
            "category" : {
                "properties" : {
                    "big" : {
                        "type": "text"
                    },
                    "medium": {
                        "type": "text"
                    },
                    "small": {
                        "type": "text"
                    }
                }
            }
        }
    }
}

template = {
    "title": "{{title}}",
    "address": "{{address}}",
    "location": {
        "lat": "{{lat}}",
        "lon": "{{lon}}"
    },
    "vector": "{{vector}}"
}



In [ ]:
from elasticsearch import Elasticsearch

# Elasticsearch 클라이언트 초기화
es = Elasticsearch("http://localhost:9200")


# 인덱스 생성
if not es.indices.exists(index=es_index_name):
    es.indices.create(index=es_index_name, body=mapping)
    print(f"Elasticsearch 인덱스 '{es_index_name}'가 생성되었습니다.")
else:
    print(f"Elasticsearch 인덱스 '{es_index_name}'는 이미 존재합니다.")

In [ ]:
from elasticsearch import helpers
import json

bulk_data = []
for doc in data_list:
    title_vector = embeddings_model.embed_query(doc[1])
    address_vector = embeddings_model.embed_query(doc[24])
    source = {
         "_index": es_index_name, 
        "_source": {
            "title": doc[1],
            "address": doc[24],
            "title_vector": title_vector,
            "address_vector": address_vector,
            "location": {"lat": doc[38], "lon": doc[37]},
            "category" : {
                "big" : doc[4],
                "medium" : doc[6],
                "small" : doc[8]
            }
        }
    }
    bulk_data.append(source)
    if len(bulk_data) == 1000:
        helpers.bulk(es, bulk_data)
        bulk_data = []
        print("inserting bulk is complete")

helpers.bulk(es, bulk_data)
bulk_data = []
    

In [ ]:
from elasticsearch import helpers
import json

vector = [item['_source']['title_vector'] for item in bulk_data]
print(len(vector[0]))

helpers.bulk(es, bulk_data)
print("inserting bulk is complete")

## 검색

In [ ]:
query_text = "마포 김밥천국"  # 검색할 텍스트
query_embedding = embeddings_model.embed_query(query_text)  # 쿼리를 벡터화
# Elasticsearch 검색 요청
search_query = {
    "_source" : ["title", "address", "location"],
    "query": {
        "script_score": {
            "query": {
                "match_all": {}  # 모든 문서에서 스코어 기반 필터링
            },
            "script": {
                "source": """
                    cosineSimilarity(params.query_vector, 'title_vector') + 
                    cosineSimilarity(params.query_vector, 'address_vector')
                """,
                "params": {
                    "query_vector": query_embedding
                }
            }
        }
    }
}
print(search_query)


# 검색 요청 실행
response = es.search(index=es_index_name, body=search_query)

# 검색 결과 출력
print("벡터 검색 결과:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])



## 광고 샘플 인덱스 생성    

### 광고 샘플 인덱스 설정 

In [ ]:
advertisement_index_name = "advertisement_poi"

advertisemment_mapping = {
    "mappings": {
        "properties": {
            "title_vector": {  # title 벡터 필드
                "type": "dense_vector",
                "dims": 1024  # Hugging Face 모델에서 생성된 벡터 차원 수
            },
            "address_vector": {  # address 벡터 필드
                "type": "dense_vector",
                "dims": 1024
            },
            "location": {  # lat, lon을 포함하는 geo_point 필드
                "type": "geo_point"
            },
            "title": {  # title 원본 텍스트
                "type": "text"
            },
            "address": {  # address 원본 텍스트
                "type": "text"
            },
            "category" : {
                "properties" : {
                    "big" : {
                        "type": "text"
                    },
                    "medium": {
                        "type": "text"
                    },
                    "small": {
                        "type": "text"
                    }
                }
            }
        }
    }
}

### 광고 인덱스 데이터 생성

In [ ]:
# reindex API를 통해 기존 인덱스의 데이터를 새 인덱스로 복사
reindex_body = {
    "source": {
        "index": "vector_poi_v2"
    },
    "dest": {
        "index": advertisement_index_name
    }
}

response = es.reindex(body=reindex_body, wait_for_completion=False, request_timeout=3600)
print("Reindex 결과:", response)

In [ ]:
from elasticsearch import Elasticsearch

# Elasticsearch 클라이언트 초기화
es = Elasticsearch("http://localhost:9200")

# 인덱스 생성
if not es.indices.exists(index=advertisement_index_name):
    es.indices.create(index=advertisement_index_name, body=advertisemment_mapping)
    print(f"Elasticsearch 인덱스 '{advertisement_index_name}'가 생성되었습니다.")
else:
    print(f"Elasticsearch 인덱스 '{advertisement_index_name}'는 이미 존재합니다.")

## User feature store

### 임의의 10명 MOCK 로그 데이터 추가

In [ ]:
from transformers import pipeline
import numpy as np
import redis
import json

# Redis 연결 설정 (예: localhost, 기본 포트 6379, DB 0)
redis_client = redis.Redis(host='localhost', port=6379, db=0)

# BAAI/bge-m3 모델로 임베딩(Feature Extraction) 파이프라인 설정
pipe = pipeline(
    "feature-extraction",
    model="BAAI/bge-m3"
)

# 10명의 사용자에 대한 예시 텍스트 (사용자 검색 로그 등)
user_texts = [
    "강남역 카페, 홍대 맛집, 스타벅스, 카페/베이커리",
    "명동 쇼핑, 코엑스, 롯데월드, 백화점",
    "서울의 밤, 한강, 남산타워, 야경",
    "부산 해운대, 해수욕, 맛집, 카페",
    "대구 맛집, 동성로, 카페, 빵집",
    "인천 차이나타운, 맛집, 문화",
    "제주도 여행, 해변, 한라산, 맛집",
    "경주 유적지, 역사, 문화, 맛집",
    "수원 화성, 전통, 맛집, 카페",
    "강원도 설악산, 자연, 트래킹, 산책"
]

# 각 사용자에 대해 임베딩 생성 후 Redis에 저장
for i, text in enumerate(user_texts, start=1):
    # 임베딩 추출: 결과는 대체로 3차원 리스트 (batch, tokens, vector_dim)
    embedding_output = pipe(text)
    # 첫 번째 배치의 모든 토큰 임베딩을 평균 풀링 (간단한 방식)
    token_embeddings = np.array(embedding_output[0])  # shape = (sequence_length, hidden_dim)
    pooled_embedding = token_embeddings.mean(axis=0)   # shape = (hidden_dim, )
    
    # 사용자 아이디 설정 (예: "user_id_1", "user_id_2", ...)
    user_id = f"user_id_{i}"
    
    # numpy 배열을 리스트로 변환 후 JSON 문자열로 변환하여 Redis에 저장
    redis_client.set(user_id, json.dumps(pooled_embedding.tolist()))
    
    print(f"{user_id} 임베딩 벡터 저장 완료. 벡터 크기: {pooled_embedding.shape}")

print("모든 사용자 임베딩 벡터가 Redis에 저장되었습니다.")